# Mental Health in the Tech Industry Data Gathering

Let's create a class for gathering the data, statistical calculation and processing data for presentation and visualization.

In [1]:
import sqlite3

In [74]:
class DataProcessing:
    ''' Class for gathering the data, statistical calculation and processing data. '''
    
    def __init__(self, path, db_name):
        self.path = path
        self.db_name = db_name
    
    def get_table(self, table):
        ''' Get table content from the database. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT * FROM {table}")
        table = c.fetchall()
        conn.close()
        return table

    def get_all_answers_per_q(self, q_id):
        ''' Get all answers from Answer table for particular question 
            represented by QuestionID number from the Question table. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT AnswerText FROM Answer WHERE QuestionID = ?", (q_id,))
        answers = set(c.fetchall())
        conn.close()
        return answers
    
    def get_users_no_per_q(self, q_id):
        ''' Get number of all answers for the question. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT count(UserID) FROM Answer WHERE QuestionID = ?", (q_id,))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    
    def get_users_no_for_q_and_answer(self, q_id, answer):
        ''' Get number of users from Answer table for particular question number (QuestionID) and answer (AnswerText). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT count(UserID) FROM Answer WHERE QuestionID = ? and AnswerText = ?", (q_id, answer))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    
    def get_users_no_per_answer(self, q_id):
        ''' Get frequency of the answers for particular question number (QuestionID) in Answer table. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT AnswerText, count(UserID) FROM Answer WHERE QuestionID = ? GROUP BY AnswerText", (q_id,))
        user_no = c.fetchall()
        conn.close()
        return user_no
    
    def get_table_based_value_from_column(self, table, column, value):
        ''' Get particular answer from provided table. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT * FROM {table} WHERE {column} = ?", (value,))
        value = c.fetchall()
        conn.close()
        return value

    def get_answers_for_q_less_occ(self, q_id, qty):
        ''' Get answers for provided question q_id where occurency is less than qty. '''
        q_id, qty = int(q_id), int(qty)
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f'SELECT AnswerText, count(UserID) as UNo FROM Answer WHERE QuestionID = ? GROUP BY AnswerText HAVING UNo < ?', 
                  (q_id, qty))
        answers = c.fetchall()
        conn.close()
        return answers

    def get_answers_for_q_greater_occ(self, q_id, qty):
        ''' Get answers for provided question q_id where occurency is greater than qty. '''
        q_id, qty = int(q_id), int(qty)
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f'SELECT AnswerText, count(UserID) as UNo FROM Answer WHERE QuestionID = ? GROUP BY AnswerText HAVING UNo > ?', 
                  (q_id, qty))
        answers = c.fetchall()
        conn.close()
        return answers
    
    def get_answers_for_q_in_year(self, q_id, year):
        ''' Get all answers from Answer table for particular question (QuestionID)
            in given year (SurveyID). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT AnswerText FROM Answer WHERE QuestionID = ? and SurveyID = ?", (q_id, year))
        answers = set(c.fetchall())
        conn.close()
        return answers
    
    def get_users_no_for_q_and_answer_in_year(self, q_id, answer, year):
        ''' Get number of users from Answer table for particular question number (QuestionID) and answer (AnswerText)
            in given year (SurveyID). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT count(UserID) FROM Answer WHERE QuestionID = ? and AnswerText = ? and SurveyID = ?", (q_id, answer, year))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    
    def get_users_no_per_q_in_year(self, q_id, year):
        ''' Get number of all answers for the question in given year (SurveyID). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT count(UserID) FROM Answer WHERE QuestionID = ? and SurveyID = ?", (q_id, year))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    
    def get_answers_for_questions(self, q_ids):
        ''' Get table content for given questions. '''
        values_no = ('?, '*len(q_ids))[:-2]
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT * FROM Answer WHERE QuestionID = {values_no}", (*q_ids))
        table = c.fetchall()
        conn.close()
        return table
    
    def get_different_answers_for_2q(self, q_id1, q_id2):
        ''' Get different answers on two questions for the same user. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"""SELECT AnswerText, UserID as uid FROM Answer 
        WHERE QuestionID = ? and AnswerText != (SELECT AnswerText FROM Answer WHERE UserID = uid and QuestionID = ?)""",
                 (q_id1, q_id2))
        table = c.fetchall()
        conn.close()
        return table
    
    def get_different_answers_for_2q_v2(self, q_id1, q_id2):
        ''' Get different answers on two questions for the same user.
            Much faster than get_different_answers_for_2q. '''
        tab_q1 = self.get_answers_ordered_by_uid(q_id1)
        tab_q2 = self.get_answers_ordered_by_uid(q_id2)
        diff_table = self._get_diff_table(tab_q1, tab_q2)
        return self._get_combined_tab_uid_based(diff_table, tab_q1)
        
    def get_answers_ordered_by_uid(self, q_id):
        ''' Get the answers ordered by UserID for given question. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT AnswerText, UserID FROM Answer WHERE QuestionID = ? Order by UserID", (q_id,))
        table = c.fetchall()
        conn.close()
        return table
    
    @staticmethod
    def _get_diff_table(tab1, tab2):
        ''' Returns the difference between two lists. '''
        table1 = set(tab1)
        table2 = set(tab2)
        return table2.difference(table1)
    
    @staticmethod
    def _get_combined_tab_uid_based(tab1, tab2):
        ''' Returns a list which combine first values from lists of tuples tab1 and tab2
            based on the second values.
            tab1 and tab2 need to have values of tuples of length 2. '''
        result_list = []
        for wc, wuid in list(tab1):
            for lc, luid in list(tab2):
                if luid == wuid:
                    result_list.append((lc, wc, wuid))
        return result_list    
    
    def get_users_no_per_two_q(self, q_id1, q_id2):
        ''' Get number of users who answered for provided two questions. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"""SELECT count(UserID), UserID as uid 
        FROM Answer WHERE QuestionID = ? and 
        EXISTS(SELECT AnswerText FROM Answer WHERE UserID = uid and QuestionID = ?)""",
                 (q_id1, q_id2))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    


In [75]:
test = DataProcessing('DB/', 'mental_health')

In [76]:
test.get_users_no_for_q_and_answer(2, 'Male')

3044

In [77]:
test.get_users_no_per_answer(2)

[('Female', 1024),
 ('Male', 3044),
 ('Non-binary', 13),
 ('Other', 107),
 ('n/a', 24),
 ('non-binary', 6)]

In [78]:
test.get_users_no_per_q(2)

4218

In [79]:
test.get_table_based_value_from_column('Answer', 'AnswerText', 'Idaho')

[('Idaho', 2014, 420, 4),
 ('Idaho', 2016, 1549, 4),
 ('Idaho', 2016, 2364, 4),
 ('Idaho', 2016, 2663, 4),
 ('Idaho', 2016, 2364, 51),
 ('Idaho', 2016, 2663, 51),
 ('Idaho', 2017, 2838, 4),
 ('Idaho', 2017, 3419, 4),
 ('Idaho', 2017, 2838, 51),
 ('Idaho', 2017, 3419, 51),
 ('Idaho', 2018, 3612, 4),
 ('Idaho', 2018, 3612, 51),
 ('Idaho', 2019, 4197, 4),
 ('Idaho', 2019, 4197, 51)]

In [80]:
test.get_answers_for_q_less_occ(2, 10)

[('non-binary', 6)]

In [81]:
test.get_answers_for_q_greater_occ(2, '10')

[('Female', 1024),
 ('Male', 3044),
 ('Non-binary', 13),
 ('Other', 107),
 ('n/a', 24)]

In [82]:
%%time
test.get_different_answers_for_2q(3, 50)

Wall time: 41 s


[('United States of America', 1511),
 ('Spain', 1566),
 ('Canada', 1569),
 ('United Kingdom', 1595),
 ('United States of America', 1647),
 ('Canada', 1688),
 ('Canada', 1702),
 ('Netherlands', 1729),
 ('Lithuania', 1777),
 ('Algeria', 1858),
 ('Pakistan', 1983),
 ('Australia', 2045),
 ('United Kingdom', 2058),
 ('Other', 2081),
 ('Romania', 2095),
 ('Japan', 2152),
 ('France', 2179),
 ('Canada', 2288),
 ('France', 2335),
 ('Germany', 2359),
 ('Italy', 2440),
 ('Italy', 2449),
 ('Taiwan', 2626),
 ('Afghanistan', 2668),
 ('Canada', 2683),
 ('United States of America', 2690),
 ('Czech Republic', 2772),
 ('Belgium', 2789),
 ('Germany', 2880),
 ('Brazil', 2883),
 ('Hungary', 2884),
 ('Greece', 2912),
 ('Ireland', 2928),
 ('Switzerland', 2954),
 ('India', 3031),
 ('Argentina', 3074),
 ('Belarus', 3120),
 ('Netherlands', 3134),
 ('France', 3140),
 ('-1', 3447),
 ('-1', 3449),
 ('Poland', 3547),
 ('New Zealand', 3683),
 ('Belgium', 3705),
 ('Germany', 3736),
 ('France', 3760),
 ('Japan', 3807)

In [83]:
%%time
test.get_different_answers_for_2q_v2(3, 50)

Wall time: 89 ms


[('Netherlands', 'United Kingdom', 1729),
 ('Canada', 'United States of America', 1702),
 ('Poland', 'United States of America', 3547),
 ('United States of America', 'Mexico', 3847),
 ('Romania', 'United States of America', 2095),
 ('Japan', 'Canada', 2152),
 ('Greece', 'Netherlands', 2912),
 ('India', 'United States of America', 3031),
 ('-1', 'n/a', 3449),
 ('France', 'United Kingdom', 2179),
 ('Spain', 'United States of America', 1566),
 ('Germany', 'United Kingdom', 2359),
 ('Italy', 'United States of America', 2440),
 ('Other', 'Germany', 2081),
 ('United Kingdom', 'Other', 3911),
 ('Canada', 'United States of America', 2683),
 ('Belarus', 'United States of America', 3120),
 ('Germany', 'Eritrea', 2880),
 ('Switzerland', 'Swaziland', 2954),
 ('Lithuania', 'United Kingdom', 1777),
 ('France', 'Switzerland', 3760),
 ('Belgium', 'Portugal', 2789),
 ('Taiwan', 'United States of America', 2626),
 ('Hungary', 'Austria', 2884),
 ('Algeria', 'United States of America', 1858),
 ('Netherlan